# دليل متكامل في **Machine Learning** — شرح تفصيلي بالعربي
مرحباً يا **ASSEM** 👋

هذا ملف **Jupyter Notebook** متكامل يشرح كل ما تحدثنا عنه — نظري + عملي + أمثلة قابلة للتشغيل خطوة بخطوة. يتضمن:
- مفاهيم أساسية (Features, Labels, Overfitting...)
- تحضير البيانات (تنظيف، ترميز، موازنة، تقليل أبعاد)
- التعلم الموجّه (انحدار وتصنيف) مع خوارزميات متعددة
- التعلم غير الموجّه (تجميع وتقليل أبعاد)
- التعلم بالتعزيز (مقدمة وQ-Learning)
- مقدمة للشبكات العصبية (مثال عملي بـ Keras)
- تقييم النماذج، ضبط المعاملات، Pipelines، ونصائح عملية

## كيفية الاستخدام
1. افتح هذا الملف في Jupyter Notebook أو JupyterLab.
2. شغّل الخلايا الخلوية (Cell) واحدة تلو الأخرى.
3. يمكنك تعديل البيانات أو المسارات لتستخدم ملفاتك المحلية.

---


## المتطلبات المسبقة
- Python 3.8+، وبيئة Jupyter
- مكتبات مقترحة: `numpy`, `pandas`, `matplotlib`, `seaborn`, `scikit-learn`, `tensorflow` (اختياري)، `joblib`

يمكن تثبيت المكتبات (إن لم تكن مثبتة) عبر:
```bash
pip install numpy pandas matplotlib seaborn scikit-learn tensorflow joblib
```

---


In [ ]:
# إعداد الاستيرادات الأساسية
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score, roc_curve, silhouette_score
)

import joblib

print('Libraries imported — ready to run examples')


## قسم 1 — تحضير البيانات (Data Preparation)

هنا سنتعلم خطوات التحضير الأساسية: فحص البيانات، تنظيفها، تعامُل مع القيم المفقودة، الترميز، والتقييس.


In [ ]:
# مثال: تحميل بيانات جاهزة (Iris) والتعرّف عليها
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
df = iris.frame
df.head()


### فحص سريع (EDA)
- `df.info()` لمعرفة أنواع الأعمدة والقيم المفقودة
- `df.describe()` للإحصاءات الوصفية
- رسومات: histogram, pairplot, heatmap للـ correlation


In [ ]:
df.info()
df.describe()

# رسم مصفوفة الارتباط
plt.figure(figsize=(8,6))
sns.heatmap(df.corr(), annot=True, fmt='.2f')
plt.title('Correlation matrix')
plt.show()


### التعامل مع القيم المفقودة
- حذف الصفوف (`dropna`) أو تعبئتها (`fillna`) بالقيمة الوسطى/المتوسطة أو نموذج توقع.
- أمثلة: `df.fillna(df.median())`


In [ ]:
# مثال افتراضي: تعبئة القيم المفقودة بالوسيط
df_missing = df.copy()
df_missing.iloc[0,0] = np.nan
print('قبل عدد القيم المفقودة:', df_missing.isna().sum().sum())
df_missing = df_missing.fillna(df_missing.median())
print('بعد التعبئة عدد القيم المفقودة:', df_missing.isna().sum().sum())


### الترميز (Encoding) للمتغيرات النصية
- `LabelEncoder` للمتغيرات الترتيبية البسيطة
- `OneHotEncoder` للفئات غير المرتبة


In [ ]:
# مثال بسيط على OneHot (لو كانت لدينا فئة)
df_cat = pd.DataFrame({'color': ['red', 'blue', 'green', 'red']})
pd.get_dummies(df_cat, columns=['color'])


### التقييس (Scaling)
- `StandardScaler` يجعل البيانات لها وسط 0 وانحراف معياري 1.
- `MinMaxScaler` يحول القيم إلى نطاق [0,1].


In [ ]:
scaler = StandardScaler()
X = df.drop(columns=['target'])
X_scaled = scaler.fit_transform(X)
pd.DataFrame(X_scaled, columns=X.columns).head()


## قسم 2 — Supervised Learning (التعلم الموجّه)
سنتناول: الانحدار (Regression) والتصنيف (Classification) مع أمثلة وكود التقييم.


### 2.1 Linear Regression (انحدار خطي)
فكرة بسيطة: نمثل العلاقة على شكل خط `y = wx + b` ونقلل مربع الفرق (MSE).


In [ ]:
# مثال عملي: انحدار خطي على بيانات مولدة
from sklearn.datasets import make_regression
X_reg, y_reg = make_regression(n_samples=200, n_features=1, noise=15, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))
print('R2:', r2_score(y_test, y_pred))


### 2.2 Logistic Regression (التصنيف الثنائي)
تستخدم دالة Sigmoid لتحويل المخرجات لقيم بين 0 و 1 ثم عتبة لتصنيفها.


In [ ]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipe = Pipeline([('scaler', StandardScaler()), ('clf', LogisticRegression(max_iter=5000))])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('F1:', f1_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix:\n', cm)


### 2.3 Decision Tree و Random Forest
- Decision Tree يعمِّل القرارات بشكل هرمي.
- Random Forest هو Ensemble من عدة أشجار يقلّل الـ overfitting.


In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print('RF Accuracy:', accuracy_score(y_test, y_pred))
feat_imp = pd.Series(rf.feature_importances_, index=data.feature_names).sort_values(ascending=False)
print('Top features:\n', feat_imp.head())


### 2.4 Support Vector Machine و KNN
- SVM يبحث عن أفضل Hyperplane لفصل الأصناف.
- KNN يعتمد على أقرب الجيران لاتخاذ القرار.


In [ ]:
svc = SVC()
svc.fit(X_train, y_train)
print('SVM Accuracy:', svc.score(X_test, y_test))

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
print('KNN Accuracy:', knn.score(X_test, y_test))


### 2.5 تقييم النموذج: مقاييس مهمة
- Regression: MSE, RMSE, MAE, R2
- Classification: Accuracy, Precision, Recall, F1, ROC-AUC, Confusion Matrix

استخدم دائماً أكثر من مقياس واحد للحكم على أداء النموذج.

### 2.6 Overfitting vs Underfitting
- **Overfitting**: أداء ممتاز على التدريب وضعيف على الاختبار. العلاج: تبسيط النموذج، regularization، جمع بيانات.
- **Underfitting**: النموذج بسيط جداً ولا يلتقط الأنماط. العلاج: استخدام نموذج أكثر تعقيداً أو ميزات أفضل.

مبدأ **Bias-Variance Tradeoff** يشرح هذا التوازن.

### 2.7 ضبط الـ Hyperparameters
- استخدم `GridSearchCV` أو `RandomizedSearchCV` لتجربة تركيبات المعاملات.
```python
param_grid = {'clf__C': [0.01, 0.1, 1, 10], 'clf__penalty': ['l2']}
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)
print(grid.best_params_, grid.best_score_)
```


## قسم 3 — Unsupervised Learning (التعلم غير الموجّه)


### 3.1 K-Means Clustering
- اختر عدد المجموعات `k` باستخدام Elbow method أو Silhouette score.


In [ ]:
from sklearn.datasets import make_blobs
X_blob, _ = make_blobs(n_samples=300, centers=4, cluster_std=0.6, random_state=42)
inertia = []
K_range = range(1,10)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_blob)
    inertia.append(kmeans.inertia_)
plt.plot(K_range, inertia, marker='o')
plt.xlabel('k')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.show()


### 3.2 PCA (تقليل الأبعاد)
- PCA يحسب المركبات الأساسية التي تشرح أكبر قدر من التباين.
```python
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_blob)
print(pca.explained_variance_ratio_)
```


In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_blob)
plt.scatter(X_pca[:,0], X_pca[:,1])
plt.title('PCA projection')
plt.show()


## قسم 4 — Reinforcement Learning (التعلم بالتعزيز)
مقدمة بسيطة ومثال على Q-Learning (مذكور في Notebook السابق أيضاً).


In [ ]:
# Q-Learning مبسّط
states = [0,1,2,3,4]
actions = [0,1]  # 0=left, 1=right
Q = np.zeros((len(states), len(actions)))
alpha = 0.1
gamma = 0.9
episodes = 200
for ep in range(episodes):
    state = 0
    while state < 4:
        action = np.random.choice([0,1])
        next_state = state + 1 if action==1 else max(0, state-1)
        reward = 1 if next_state==4 else 0
        Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state, action])
        state = next_state
Q


## قسم 5 — مقدمة للشبكات العصبية (Neural Networks)
سأقدّم مثال بسيط باستخدام `tensorflow.keras` لإنشاء MLP (شبكة متعدّدة الطبقات) لتصنيف بيانات سرطان الثدي.


In [ ]:
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout
    # إعداد البيانات نفسها المستخدمة أعلاه
    X_train_s = StandardScaler().fit_transform(X_train)
    X_test_s = StandardScaler().fit_transform(X_test)
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train_s.shape[1],)),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    print('Keras model created (train separately if you want to run).')
except Exception as e:
    print('TensorFlow not available or another issue — model cell left as example.\n', e)


## قسم 6 — نشر النماذج (Deployment) وجوانب عملية
- حفظ النموذج: `joblib.dump(model, 'model.joblib')`
- استخدام `Pipeline` لتجميع خطوات التحضير + النموذج حتى يسهل النشر.
- أفكار للنشر: Flask/FastAPI لخدمة نموذج عبر REST، أو تحويل نموذج TensorFlow إلى SavedModel.


In [ ]:
# مثال: حفظ نموذج RandomForest
joblib.dump(rf, '/mnt/data/rf_breast_cancer.joblib')
print('Model saved to /mnt/data/rf_breast_cancer.joblib')


## قسم 7 — نصائح عملية ونهائية
- ابدأ دائماً بـ EDA وVisualization.
- قسّم البيانات جيداً (train/validation/test) أو استخدم cross-validation.
- حافظ على Reproducibility (random_state, seeds).
- سجل تجاربك (مثل MLflow أو حتى ملف CSV بسيط) لتتبع الأداء.
- جرّب طرق Explainability (SHAP, LIME) إذا كنت بحاجة لتفسير قرارات النموذج.


## تمارين واقتراحات تطبيقية
1. تطبيق الانحدار الخطي على مجموعة بيانات أسعار المنازل (California Housing).
2. بناء نموذج تصنيف (RandomForest) لمجموعة بيانات Titanic أو Breast Cancer.
3. تجربة K-Means على بيانات العملاء ومحاولة تفسير المجموعات.
4. تنفيذ GridSearchCV للعثور على أفضل hyperparameters.


---
### انتهى الدليل — هل تريد نسخة PDF أو ملف Markdown قابل للطباعة؟
أو هل تريد أن أعدّ نسخة مبسطة للـ Slides (عرض تقديمي) يمكنك مشاركته؟
